### Database Setup and Connection

This cell performs three essential tasks for our NL2SQL system:

1. **Establishes Database Connection**: Connects to an SQLite database file containing employee information using LangChain's `SQLDatabase` wrapper
2. **Validates Connection**: Uses a try-except block to verify the connection is successful and retrieves available table names
3. **Retrieves Schema Information**: Fetches the complete database schema (table structures, column names, data types, relationships) which will be used later to generate accurate SQL queries

**Why Schema Matters**: The schema provides crucial context for the LLM to understand what data is available and how to construct valid SQL queries.


In [1]:
from langchain_community.utilities import SQLDatabase
from langchain_core.tools import tool

print("Setting up employees database ...")

db = SQLDatabase.from_uri("sqlite:///employees_db-full-1.0.6.db")

# Check the connection and get basic info
try:
    tables = db.get_usable_table_names()
    print(f"Database connected successfully")
    print(f"Found {len(tables)} tables: {', '.join(tables)}")
except Exception as e:
    print(f"Database connection failed: {e}")
    
SCHEMA = db.get_table_info()
print("Database schema:\n", SCHEMA)


Setting up employees database ...
Database connected successfully
Found 6 tables: departments, dept_emp, dept_manager, employees, salaries, titles
Database schema:
 
CREATE TABLE departments (
	dept_no CHAR(4) NOT NULL, 
	dept_name VARCHAR(40) NOT NULL, 
	PRIMARY KEY (dept_no), 
	UNIQUE (dept_name)
)

/*
3 rows from departments table:
dept_no	dept_name
d009	Customer Service
d005	Development
d002	Finance
*/


CREATE TABLE dept_emp (
	emp_no INTEGER NOT NULL, 
	dept_no CHAR(4) NOT NULL, 
	from_date DATE NOT NULL, 
	to_date DATE NOT NULL, 
	PRIMARY KEY (emp_no, dept_no), 
	FOREIGN KEY(dept_no) REFERENCES departments (dept_no), 
	FOREIGN KEY(emp_no) REFERENCES employees (emp_no)
)

/*
3 rows from dept_emp table:
emp_no	dept_no	from_date	to_date
10001	d005	1986-06-26	9999-01-01
10002	d007	1996-08-03	9999-01-01
10003	d004	1995-12-03	9999-01-01
*/


CREATE TABLE dept_manager (
	dept_no CHAR(4) NOT NULL, 
	emp_no INTEGER NOT NULL, 
	from_date DATE NOT NULL, 
	to_date DATE NOT NULL, 
	PRIMARY K

### LLM Configuration and Chain Setup

This cell initializes our Language Model and creates a basic processing chain:

**LLM Configuration**:
- **Model**: Uses Llama 3.2 running locally via Ollama (requires Ollama to be running on port 11434)
- **Temperature = 0**: Ensures deterministic, consistent outputs - crucial for SQL generation where we need precise, repeatable queries
- **num_predict = 256**: Limits the response length to 256 tokens

**Chain Components**:
1. **ChatPromptTemplate**: Formats the input prompt
2. **LLM (ChatOllama)**: Processes the prompt and generates responses
3. **StrOutputParser**: Extracts the text response from the LLM output

**Test Query**: A simple invocation to verify the chain is working before building the complete NL2SQL pipeline.


In [2]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import (    
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    ChatPromptTemplate                                                                     
)
from langchain_core.output_parsers import StrOutputParser

# Ollama configuration
base_url = "http://localhost:11434"
model = 'llama3.2:latest'

# Initialize LLM with deterministic settings for SQL generation
llm = ChatOllama(
    base_url=base_url,
    model=model,
    temperature=0,  # Deterministic output - important for consistent SQL generation
    num_predict=512,  # Increased for longer SQL queries
)

# Create a simple test chain
template = ChatPromptTemplate.from_template("{prompt}")
chain = template | llm | StrOutputParser()

# Test the chain with a sample query
test_query = "What is the capital city of France?"
response = chain.invoke({'prompt': test_query})
print("LLM Response:", response)


LLM Response: The capital city of France is Paris.


### `get_database_schema` – View table schemas

This tool returns the **database schema** so the model (and the user) knows which tables and columns exist.

**How it works:**
- If a `table_name` is provided:
  - It first gets the list of usable tables from the database.
  - It checks (case-insensitive) whether `table_name` exists.
  - If the table exists, it returns the schema for **only that table**.
  - If the table does not exist, it returns an error message listing available tables.
- If `table_name` is empty or not provided:
  - It returns the **full database schema** stored in the global `SCHEMA` variable.

**Why it matters in NL2SQL:**
- The schema tells the LLM what tables and columns are available.
- Good NL2SQL performance depends on using the correct table and column names.


In [3]:
@tool
def get_database_schema(table_name:str = None):
    """
    Retrieves the schema for a specific table or all tables in the database.
    
    Args:
        table_name (str, optional): Name of the table to get schema for. 
                                    If None or empty, returns schema for all tables.
    
    Returns:
        str: The table schema or an error message.
    """
    print(f"Getting schema for : {table_name if table_name else 'all tables'}")
    
    if table_name:
        try:
            tables = db.get_usable_table_names()
            if table_name.lower() in [t.lower() for t in tables]:
                result = db.get_table_info([table_name])
                print(f"Retrieved schema for table: {table_name}")
                return result
            else:
                return f"Error: Table '{table_name}'not found. Available tables: {','.join(tables)}"        
        except Exception as e:
            return f"Error getting table info: {e}"
    
    else:
        print("retried full database schema")
        return SCHEMA
            
                    
                
    

### `generate_sql_query` – Turn questions into SQL

This tool converts a **natural language question** into a **SQL SELECT query** using the database schema.

**Inputs:**
- `question` (str): The user’s natural language question.
- `schema_info` (str, optional): A specific schema snippet (e.g., for one table). If not provided, it uses the global `SCHEMA`.

**How it works:**
- It chooses which schema to use:
  - If `schema_info` is passed, it uses that.
  - Otherwise, it uses the full `SCHEMA`.
- It constructs a detailed prompt that:
  - Shows the schema.
  - Asks the model to generate a SQL query that answers the question.
  - Enforces rules:
    - Only `SELECT` statements.
    - Only existing tables and columns.
    - Use `WHERE`, `GROUP BY`, `ORDER BY` when needed.
    - Limit results to 10 rows unless stated otherwise.
    - Use valid SQLite syntax.
- It sends this prompt to the LLM and returns the **raw SQL query** (with surrounding whitespace removed).

**Why it matters in NL2SQL:**
- This is the **core generation step**: it transforms user intent into an executable SQL query.
- The rules in the prompt help reduce invalid or dangerous SQL.


In [4]:
@tool 
def generate_sql_query(question: str, schema_info:str = None):
    """
    generate a sql SELECT query from a natural language question using database schema
    always use this after getting schema information.
    """
    print(f"Genarating sql for : {question[:100]}...")
    
    schema_to_use = schema_info if schema_info else SCHEMA
    
    prompt = f"""
                based on this database schema:
                {schema_to_use}
                
                Generate a SQL query to answer this question: {question}
                
                Rules:
                - Use only SELECT statements
                - Include only existing columns and tables
                - Add appropriate WHERE, GROUP BY, ORDER BY clauses as needed
                - Limit results to 10 rows unless specified otherwise
                - Use proper SQL syntax for SQLite
                
                Return only the SQL query, nothing else.
                """
    try:
        response = llm.invoke(prompt)
        query = response.content.strip()
        print(f"Generated SQL query")
        return query
    except Exception as e:
        return f"Error generating query: {e}"
    
    
    
    

### `validate_sql_query` – Check SQL safety and syntax style

This tool checks whether a generated SQL query is **safe** and follows basic constraints before execution.

**Inputs:**
- `query` (str): The SQL query (possibly with markdown formatting, like ```

**How it works:**
1. **Cleanup:**
   - Strips leading/trailing spaces.
   - Removes ```sql and ```
   - Removes a trailing semicolon.
2. **Statement type check:**
   - If the cleaned query does **not** start with `SELECT` (case-insensitive), it returns:
     - `"Error: Only SELECT statements are allowed"`.
3. **Dangerous keyword filter:**
   - Converts the query to uppercase and scans for:
     - `INSERT`, `UPDATE`, `DELETE`, `ALTER`, `DROP`, `CREATE`, `TRUNCATE`.
   - If any appear, it returns an error like:
     - `"Error: DROP operations are not allowed!"`.
4. If all checks pass:
   - It prints that validation passed.
   - It returns `"valid: <clean_query>"`.

**Why it matters in NL2SQL:**
- Prevents execution of **write / destructive operations**.
- Ensures the system only runs **read-only SELECT** queries.
- Acts as a safety gate between LLM output and the database.


In [19]:
import regex as re

In [5]:
@tool
def validate_sql_query(query:str):
    """
    Validate SQL query for safety and syntax before execution.
    returns 'valid:<query>' if safe or 'error: <message>' if unsafe.
    """
    print(f"validating SQL: {query[:100]}...")
    
    
    clean_query = query.strip()
    
    clean_query = re.sub(r'```sql\s*', '', clean_query, flags = re.IGNORECASE)
    clean_query = re.sub(r'```\s*', '', clean_query)
    clean_query = clean_query.strip().rstrip(";")
    
    if not clean_query.lower().startswith("select"):
        return "Error: Only SELECT statements are allowed"
    
    dangerous_keywords = ['INSERT', 'UPDATE', 'DELETE', 'ALTER', 'DROP', 'CREATE', 'TRUNCATE']
    query_upper = clean_query.upper()
    
    for keyword in dangerous_keywords:
        if keyword in query_upper:
            return f"Error: {keyword} operations are not allowed!"
    print("Query validation passed")
    return f"valid: {clean_query}"
        
    


### `execute_sql_query` – Run a safe SQL query

This tool executes a **validated** SQL query against the database and returns the results.

**Inputs:**
- `query` (str): The SQL query, usually the output of `validate_sql_query`.

**How it works:**
1. **Preprocessing:**
   - Strips whitespace.
   - If the string starts with `"valid: "`, it removes that prefix and keeps the raw SQL.
   - Removes ```sql and ```
   - Strips again and removes a trailing semicolon.
2. **Execution:**
   - Uses `db.run(clean_query)` to execute the SQL.
3. **Result handling:**
   - If there is a result (e.g., rows returned), it returns:
     - `"Query results\n{result}"`.
   - If there are no rows, it returns:
     - `"Query executed successfully but returned no results"`.
4. **Error handling:**
   - If an exception occurs, it prints and returns a message like:
     - `"Execution Error: <error_message>"`.

**Why it matters in NL2SQL:**
- This is the step where the system actually **interacts with the database**.
- It ensures the query passed through validation and then returns user-readable results.


In [6]:
@tool 
def execute_sql_query(query:str):
    """
    Exectire a validated sql query and return results.
    Only use this after validating the query for safety.
    """
    print(f"Executing SQL: {query[:100]}...")
    
    try:
        clean_query = query.strip()
        if clean_query.startswith("valid: "):
            clean_query = clean_query[7:]
            
        clean_query = re.sub(r'```sql\s*', '', clean_query, flags = re.IGNORECASE)
        clean_query = re.sub(r'```\s*', '', clean_query)
        clean_query = clean_query.strip().rstrip(";")
        
        result = db.run(clean_query)
        print("Query executed successfully")
        
        if result:
            return f"Query results\n{result}"
        else:
            return "Query executed succcesfully but returned no reuslts"
        
    except Exception as e:
        error_msg = f"Execution Error : {str(e)}"
        print(f' {error_msg}')
        return error_msg

### `fix_sql_error` – Repair failing SQL queries

This tool uses the LLM to **debug and fix** SQL queries that failed during validation or execution.

**Inputs:**
- `original_query` (str): The SQL query that failed.
- `error_message` (str): The exact error obtained from validation or execution.
- `question` (str): The original natural language question from the user.

**How it works:**
- It builds a detailed prompt that includes:
  - The failing query.
  - The error message.
  - The original user question.
  - The full database schema (`SCHEMA`).
- It instructs the LLM to:
  - Analyze why the query failed.
  - Produce a **corrected SQL query** that:
    - Fixes the specific error.
    - Still answers the original question.
    - Uses only valid tables and columns from the schema.
    - Follows SQLite syntax.
  - Return **only** the corrected SQL query (no explanation text).
- It calls the LLM with this prompt and returns the cleaned, fixed query.

**Why it matters in NL2SQL:**
- Automatically recovers from many common errors (wrong column names, bad joins, syntax issues).
- Reduces manual debugging effort and makes the system more **robust** for end users.


In [7]:
@tool
def fix_sql_error(original_query:str, error_message:str, question:str):
    """Fix a failed SQL by analyzing the error and generating a corrected version.
    Use this when validation or exectuion fails."""
    print(f" Fixing SQL error: {error_message[:100]}...")
    
    fix_prompt = f"""
                    The following SQL query failed:
                    Query: {original_query}
                    Error: {error_message}
                    Original Question: {question}
                    
                    Database Schema:
                    {SCHEMA}
                    
                    Analyze the error and provide a corrected SQL query that:
                    1. Fixes the specific error mentioned
                    2. Still answers the original questions
                    3. Uses only valid table and column names from the schema
                    4. Follows SQLite syntax rules
                    
                    Return only the corrected SQL query, nothing else.
                    
    """
    
    
    try:
        response = llm.invoke(fix_prompt)
        fixed_query = response.content.strip()
        print("Generated fixed SQL query")
        return fixed_query
    
    except Exception as e:
        return f"Error generating fix: {e}"

### Define the Workflow State

The `SQLState` is a typed dictionary that holds all the information as it flows through our graph. Each node can read from and write to this state.

**State Variables:**
- `question`: The user's natural language question
- `sql_query`: The generated SQL query
- `validation_result`: Result from the validation check
- `execution_result`: Results from executing the query
- `final_answer`: The formatted answer for the user
- `error_count`: Tracks how many times we've tried to fix errors
- `error_message`: Stores the current error (if any)


In [8]:
from langgraph.graph import StateGraph, END
from typing import TypedDict

In [9]:
class SQLState(TypedDict):
    question: str
    sql_query: str
    validation_result: str
    execution_result: str
    final_answer: str
    error_count: int
    error_message: str


### Workflow Nodes: SQL Generation and Validation

**Nodes** are the individual steps in our workflow. Each node:
- Receives the current state
- Performs one specific task
- Updates the state
- Returns the modified state

**Node 1 - Generate:** Converts natural language to SQL using our `generate_sql_query` tool

**Node 2 - Validate:** Checks the SQL for safety (no DELETE, DROP, etc.) using our `validate_sql_query` tool


In [10]:
# Node 1: Call your generate_sql_query tool
def generate_node(state: SQLState) -> SQLState:
    """Generate SQL query from natural language question"""
    print(f"\n🔄 Step 1: Generating SQL...")
    
    # Call the tool with the user's question
    result = generate_sql_query.invoke({
        "question": state["question"], 
        "schema_info": None
    })
    
    # Store the generated SQL in state
    state["sql_query"] = result
    return state


# Node 2: Call your validate_sql_query tool
def validate_node(state: SQLState) -> SQLState:
    """Validate SQL query for safety before execution"""
    print(f"\n🔄 Step 2: Validating SQL...")
    
    # Call the validation tool
    result = validate_sql_query.invoke({"query": state["sql_query"]})
    state["validation_result"] = result
    
    # Track if there's an error for conditional routing
    if result.startswith("Error"):
        state["error_message"] = result
    else:
        state["error_message"] = ""
    
    return state


### Workflow Nodes: Execution and Error Recovery

**Node 3 - Execute:** Runs the validated SQL query against the database

**Node 4 - Fix:** If execution fails, this node uses the LLM to analyze the error and generate a corrected SQL query


In [11]:
# Node 3: Call your execute_sql_query tool
def execute_node(state: SQLState) -> SQLState:
    """Execute the validated SQL query"""
    print(f"\n🔄 Step 3: Executing SQL...")
    
    # Skip execution if validation failed
    if state["validation_result"].startswith("Error"):
        state["execution_result"] = "Skipped due to validation error"
        return state
    
    # Execute the query
    result = execute_sql_query.invoke({"query": state["validation_result"]})
    state["execution_result"] = result
    
    # Check if execution had errors
    if "Error" in result or "error" in result:
        state["error_message"] = result
    
    return state


# Node 4: Call your fix_sql_error tool
def fix_node(state: SQLState) -> SQLState:
    """Fix SQL query when errors occur"""
    print(f"\n🔄 Step 4: Fixing SQL (attempt {state['error_count'] + 1})...")
    
    # Get the current error from execution_result
    error_msg = state.get("execution_result", state.get("error_message", ""))
    
    # Call the fix tool with error context
    result = fix_sql_error.invoke({
        "original_query": state["sql_query"],
        "error_message": error_msg,
        "question": state["question"]
    })
    
    # Update state with fixed query and increment retry counter
    state["sql_query"] = result
    state["error_count"] = state.get("error_count", 0) + 1
    
    # Clear previous results for fresh validation/execution
    state["validation_result"] = ""
    state["execution_result"] = ""
    state["error_message"] = ""
    
    return state


### Workflow Node: Answer Formatting

**Node 5 - Answer:** Formats the final response for the user. This could be enhanced to use the LLM to create natural language answers from raw SQL results.


In [12]:
# Node 5: Format the final answer
def answer_node(state: SQLState) -> SQLState:
    """Format the final answer for the user"""
    print(f"\n🔄 Step 5: Formatting answer...")
    
    execution_result = state.get("execution_result", "")
    question = state["question"]
    sql_query = state["sql_query"]
    
    # Check for errors
    if "Error" in execution_result or "error" in execution_result:
        state["final_answer"] = f"Query failed:\n\n{execution_result}"
        return state
    
    # Option 1: Simple formatting (current)
    # state["final_answer"] = f"Results:\n{execution_result}"
    
    # Option 2: Natural language formatting using LLM
    answer_prompt = f"""
You are a helpful database assistant. Provide a clear, natural language answer.

User Question: {question}
SQL Query: {sql_query}
Query Results: {execution_result}

Provide a brief, natural language answer (2-3 sentences). 
Format numbers nicely (e.g., use commas for large numbers).
"""
    
    try:
        response = llm.invoke(answer_prompt)
        state["final_answer"] = response.content.strip()
    except Exception as e:
        # Fallback to raw results if LLM fails
        state["final_answer"] = f"Results:\n{execution_result}"
    
    return state


### Conditional Routing: Retry Logic

This function determines whether to retry after an error or proceed to the final answer.

**Logic:**
- If there's an error AND we haven't exceeded 3 attempts → go to "fix" node
- If we've tried 3 times OR there's no error → go to "answer" node

This is called a **conditional edge** - the next node depends on the current state.


In [13]:
def should_retry(state: SQLState) -> str:
    """Decide whether to retry after an error or proceed to answer"""
    error_count = state.get("error_count", 0)
    execution_result = state.get("execution_result", "")
    
    # Check if there was an execution error
    has_error = "Error" in execution_result or "error" in execution_result
    
    if has_error and error_count < 3:
        print(f"🔁 Error detected, retrying (attempt {error_count + 1}/3)...")
        return "fix"
    elif has_error and error_count >= 3:
        print(f"⛔ Max retries reached, proceeding to answer")
        return "answer"
    else:
        print(f"✅ Success! Proceeding to answer")
        return "answer"


### Construct the LangGraph Workflow

Now we assemble all the nodes and edges into a directed graph that defines our NL2SQL pipeline.

**Graph Structure:**



**Edge Types:**
- **Regular edges** (→): Always follow this path
- **Conditional edges** (⚡): Choose path based on state


In [14]:
# Initialize the graph with our state schema
workflow = StateGraph(SQLState)

# Add all nodes to the graph
workflow.add_node("generate", generate_node)
workflow.add_node("validate", validate_node)
workflow.add_node("execute", execute_node)
workflow.add_node("fix", fix_node)
workflow.add_node("answer", answer_node)

# Define the workflow edges
workflow.set_entry_point("generate")  # Start here
workflow.add_edge("generate", "validate")  # Generate → Validate
workflow.add_edge("validate", "execute")   # Validate → Execute

# Conditional edge: Execute → Fix OR Answer (based on should_retry function)
workflow.add_conditional_edges(
    "execute", 
    should_retry, 
    {
        "fix": "fix",      # If should_retry returns "fix"
        "answer": "answer"  # If should_retry returns "answer"
    }
)

workflow.add_edge("fix", "validate")  # Fix → Validate (retry loop)
workflow.add_edge("answer", END)      # Answer → END (terminate)

# Compile the graph into an executable workflow
sql_graph = workflow.compile()

print("✅ NL2SQL workflow graph compiled successfully!")


✅ NL2SQL workflow graph compiled successfully!


### Query Interface Function

This function provides a simple interface for users to ask questions in natural language.

**It handles:**
- Initializing the state with the user's question
- Running the graph
- Displaying the formatted answer


In [16]:
def ask_sql(question: str):
    """
    Ask a natural language question about the database.
    
    Args:
        question (str): Natural language question
        
    Returns:
        dict: The final state containing the answer and all intermediate results
    """
    print(f"\n{'='*80}")
    print(f"QUESTION: {question}")
    print(f"{'='*80}")
    
    # Initialize the state
    initial_state = {
        "question": question,
        "sql_query": "",
        "validation_result": "",
        "execution_result": "",
        "final_answer": "",
        "error_count": 0,
        "error_message": ""
    }
    
    # Run the graph
    result = sql_graph.invoke(initial_state)
    
    # Display the answer
    print(f"\n{'='*80}")
    print(f"ANSWER:")
    print(f"{'='*80}")
    print(result["final_answer"])
    
    return result


In [18]:
ask_sql("What is the highest salary in the database?")



QUESTION: What is the highest salary in the database?

🔄 Step 1: Generating SQL...
Genarating sql for : What is the highest salary in the database?...
Generated SQL query

🔄 Step 2: Validating SQL...
validating SQL: SELECT MAX(salary) FROM salaries...
Query validation passed

🔄 Step 3: Executing SQL...
Executing SQL: valid: SELECT MAX(salary) FROM salaries...
Query executed successfully
✅ Success! Proceeding to answer

🔄 Step 5: Formatting answer...

ANSWER:
The highest salary in the database is $158,220. This figure represents the maximum value of the 'salary' column in the 'salaries' table. It's worth noting that this salary belongs to a specific individual or entity within the database.


{'question': 'What is the highest salary in the database?',
 'sql_query': 'SELECT MAX(salary) FROM salaries',
 'validation_result': 'valid: SELECT MAX(salary) FROM salaries',
 'execution_result': 'Query results\n[(158220,)]',
 'final_answer': "The highest salary in the database is $158,220. This figure represents the maximum value of the 'salary' column in the 'salaries' table. It's worth noting that this salary belongs to a specific individual or entity within the database.",
 'error_count': 0,
 'error_message': ''}